## Read and Save Query Status in CSV for all Workflows

<br>

**Read all the JSON files for all the workflows and print out the messages and query status to a CSV file**

<br>

In [1]:
# import all the modules. NB: submit_run_ars_modules contains all the modules to submit job to ARAX

import json
import requests
from gamma_viewer import GammaViewer
from IPython.display import display
#from submit_run_ars_modules import submit_to_ars, submit_to_devars, printjson, retrieve_devars_results
import glob 
import os
from collections import defaultdict
import pandas as pd
from time import sleep
from os import path
from datetime import datetime



In [2]:
def submit_to_ars(m,ars_url='https://ars.ci.transltr.io/ars/api',arax_url='https://arax.ncats.io'):
    submit_url=f'{ars_url}/submit'
    response = requests.post(submit_url,json=m)
    try:
        message_id = response.json()['pk']
    except:
        print('fail')
        message_id = None
    print(f'{arax_url}/?source=ARS&id={message_id}')
    return message_id

##https://ars.ci.transltr.io/ars/api

def retrieve_ars_results(mid,ars_url='https://ars.ci.transltr.io/ars/api'):
    pk = 'https://arax.ncats.io/?source=ARS&id=' + mid
    message_url = f'{ars_url}/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    print( j['status'] )
    results = {}
    dictionary = {}
    for child in j['children']:
        print(child['status'])
        if child['status']  == 'Done':
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except Exception as e:
                nresults=0
                child['status'] = 'ARS Error'
        elif child['status'] == 'Error':
            nresults=0
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except Exception as e:
                #print(e)
                child['status'] = 'ARS Error'
        else:
            nresults = 0
            
        dictionary['pk_id'] =  pk  
            
        if ((child['status'] == 'Done') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'No Results'
            #test =  [child['actor']['agent'], 'No Results']
        elif ((child['status'] == 'ARS Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'ARS Error'
        elif ((child['status'] == 'Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'Error'
            #test =  [child['actor']['agent'], 'ARS Error']
        elif ((child['status'] == 'Done') & (nresults != 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Results'
        elif ((child['status'] == 'Unknown') & (nresults == 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Unknown'
        
        
        print(child['actor']['agent'], child['status'], nresults)
        #test =  [child['actor']['agent'], child['status'], nresults]
        #test2.append(test)
    return dictionary


#def submit_to_devars(m):
#    return submit_to_ars(m,ars_url='https://ars-dev.transltr.io/ars/api',arax_url='https://arax.ncats.io')

#def retrieve_devars_results(m):
#     return retrieve_ars_results(m,ars_url='https://ars-dev.transltr.io/ars/api')

def printjson(j):
    print(json.dumps(j,indent=4))
    
def make_hyperlink(value):
    return '=HYPERLINK("%s", "%s")' % (value.format(value), value)

<br>

**The below code reads each JSON files from the Workflows A through D (subdirectories). The queries are submitted to ARAX and output is saved in a dictionary, where the key is the file name of the JSON to denote which query is being run and the values assigned to the key is the query id**

<br>


In [3]:
PATH = r'/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo'
EXT = "*.json"
dict_workflows = {}
for root, dirs, files in os.walk(PATH): # step 1: accessing file
    #print(root)
    for name in files:
        
        if name.endswith((".json")):
            file_read = path.join(root, name)
            dir_name = (os.path.splitext(os.path.basename(root))[0])
            print(file_read)
            
            filename = (os.path.splitext(os.path.basename(file_read))[0])
            print(filename)
            with open(file_read,'r') as inf:
                query = json.load(inf)
                
                kcresult = submit_to_ars(query)
                
                result_status = retrieve_ars_results(kcresult)
                
        
                dict_workflows[filename] = kcresult
                            
                sleep(200)

/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.3_KCNMA1.json
A.3_KCNMA1
https://arax.ncats.io/?source=ARS&id=e585225c-c201-46e1-8798-f3d5da8d5bc0
Running
Done
ara-explanatory Done 0
Running
ara-unsecret Running 0
Running
ara-bte Running 0
Running
kp-openpredict Running 0
Running
kp-textmining Running 0
Done
ara-improving Done 5
Running
ara-arax Running 0
Running
ara-aragorn Running 0
Running
kp-molecular Running 0
Running
kp-cam Running 0
Done
kp-icees Done 0
Done
kp-genetics Done 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.8_EGFR_simple.json
A.8_EGFR_simple
https://arax.ncats.io/?source=ARS&id=56f06fb4-315a-4d4b-a7f7-5a18e7119b65
Running
Done
kp-genetics Done 0
Running
kp-textmining Running 0
Running
kp-chp Running 0
Done
kp-molecular Done 0
Done
kp-icees Done 0
Running
kp-cam Running 0
Running
ara-unsecret Running 0
Running
kp-openpredict Running 0
Error
ara-aragorn Error 0
Running
ara-bte Running 0
Done
ara-explanatory Done 0
Ru

Running
Running
kp-cam Running 0
Done
kp-icees ARS Error 0
Running
kp-textmining Running 0
Error
kp-cohd Error 0
Running
ara-unsecret Running 0
Running
ara-arax Running 0
Error
kp-chp Error 0
Running
kp-molecular Running 0
Unknown
kp-genetics Unknown 0
Running
ara-aragorn Running 0
Running
ara-bte Running 0
Done
ara-improving ARS Error 0
Done
kp-openpredict Done 0
Done
ara-explanatory Done 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowB/B.1d_DILI_three-hop-from-disease-or-phenotypic-feature_trapi.json
B.1d_DILI_three-hop-from-disease-or-phenotypic-feature_trapi
https://arax.ncats.io/?source=ARS&id=d97ca0f0-1c11-45ba-9d0b-c4732f35df59
Running
Running
ara-improving Running 0
Running
ara-unsecret Running 0
Running
kp-molecular Running 0
Running
kp-openpredict Running 0
Error
kp-cohd Error 0
Running
kp-cam Running 0
Done
ara-explanatory Done 0
Running
kp-textmining Running 0
Error
kp-chp Error 0
Running
kp-icees Running 0
Unknown
kp-genetics Unknown 0
Running
ara-ar

<br>

### Codes below are for recording messages and generating outout as csv

<br>

In [4]:
workflow_result_messages = {}
for keys, val in dict_workflows.items():
    print(keys, val)
    
    result_status = retrieve_ars_results(val)
    
    workflow_result_messages[keys] = result_status

A.3_KCNMA1 e585225c-c201-46e1-8798-f3d5da8d5bc0
Done
Done
ara-explanatory Done 0
Error
ara-unsecret Error 0
Done
ara-bte Done 40
Done
kp-openpredict Done 0
Done
kp-cohd ARS Error 0
Done
kp-textmining Done 0
Done
ara-improving Done 5
Done
ara-arax Done 68
Done
ara-aragorn Done 341
Done
kp-molecular Done 99
Done
kp-cam Done 0
Done
kp-icees Done 0
Done
kp-chp Done 10
Done
kp-genetics Done 0
A.8_EGFR_simple 56f06fb4-315a-4d4b-a7f7-5a18e7119b65
Done
Done
kp-genetics Done 0
Done
kp-textmining Done 0
Done
kp-chp Done 0
Done
kp-molecular Done 0
Done
kp-icees Done 0
Done
kp-cam Done 0
Error
ara-unsecret Error 0
Done
kp-openpredict Done 0
Error
ara-aragorn Error 0
Done
ara-bte Done 0
Done
ara-explanatory Done 0
Done
ara-arax Done 336
Error
kp-cohd Error 0
Done
ara-improving ARS Error 0
A.0_RHOBTB2_direct 28e96df2-53ca-491f-a468-b1a5de653473
Done
Done
kp-chp Done 0
Error
ara-unsecret Error 0
Done
kp-genetics Done 0
Done
kp-textmining Done 0
Done
kp-icees Done 0
Done
ara-explanatory Done 0
Done
ar

kp-openpredict Done 0
Error
ara-unsecret Error 0
Error
ara-arax Error 0
Done
ara-explanatory Done 0
Error
kp-molecular Error 0
Error
kp-cohd Error 0
Error
kp-textmining Error 0
Error
ara-bte Error 0
Done
ara-improving Done 166
Error
ara-aragorn Error 0
Unknown
kp-genetics Unknown 0
D.2_ssri-heart-disease 3e00623c-9bde-4519-a93d-293099d44137
Done
Done
ara-arax Done 500
Error
ara-bte Error 0
Unknown
kp-genetics Unknown 0
Error
ara-unsecret Error 0
Done
kp-chp Done 0
Done
ara-improving Done 6
Error
kp-cam Error 0
Done
kp-icees ARS Error 0
Done
ara-aragorn Done 121
Done
ara-explanatory Done 0
Done
kp-openpredict Done 0
Error
kp-molecular Error 0
Error
kp-textmining Error 0
Error
kp-cohd Error 0
C.2_ChemSubstances_related_to_GeneSet 8c2558d9-b8d0-4140-84cf-967f145b89ae
Done
Error
ara-arax Error 0
Error
kp-textmining Error 0
Error
ara-improving Error 0
Done
kp-icees ARS Error 0
Unknown
kp-genetics Unknown 0
Error
kp-cohd Error 0
Done
ara-explanatory Done 0
Error
ara-unsecret Error 0
Error
kp

In [5]:
## Convert mesages to a dataframe
col = []
final_dict = defaultdict(list)
for k in sorted(workflow_result_messages):
    print(k)
    col.append(k)
    
    for key, value in workflow_result_messages[k].items():
#         if key.startswith('kp-'):
#             key_mod = key.replace('kp-','')
#         else:
#             key_mod = key
        
        final_dict[key].append(value)

    final_dict = dict(final_dict)
    
df = pd.DataFrame(final_dict).T
df.rename(columns=dict(zip(df.columns, col)), inplace=True)

A.0_RHOBTB2_direct
A.1_RHOBTB2
A.2_RHOBTB2_twohop
A.3_KCNMA1
A.8_EGFR_simple
A.9_EGFR_advanced
B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi
B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi
B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi
B.1d_DILI_three-hop-from-disease-or-phenotypic-feature_trapi
B.1e_DILI_three-hop-from-disease-or-phenotypic-feature_trapi
B.1f_DILI-three-hop-from-disease-or-phenotypic-feature_trapi
B.2_DILI-fourth-one-hop-from-disease-or-phenotypic-feature_trapi
B.3_DILI-one-hop-from-genes
B.4_one-hop-gene-biological-process-or-activity_trapi
C.1_ChemSubstances_related_to_Disease
C.2_ChemSubstances_related_to_GeneSet
D.1_parkinsons-crohns
D.2_ssri-heart-disease
D.3_ssri-heart-disease-one-hop
D.4_tryptophan-kynurenine-with-curie
D.5_tryptophan-kynurenine
D.6_metformin-ferritin


In [6]:
df

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1d_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,...,B.3_DILI-one-hop-from-genes,B.4_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,C.2_ChemSubstances_related_to_GeneSet,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine-with-curie,D.5_tryptophan-kynurenine,D.6_metformin-ferritin
pk_id,https://arax.ncats.io/?source=ARS&id=28e96df2-...,https://arax.ncats.io/?source=ARS&id=f1695a43-...,https://arax.ncats.io/?source=ARS&id=8c63ec23-...,https://arax.ncats.io/?source=ARS&id=e585225c-...,https://arax.ncats.io/?source=ARS&id=56f06fb4-...,https://arax.ncats.io/?source=ARS&id=0e016263-...,https://arax.ncats.io/?source=ARS&id=c6856c98-...,https://arax.ncats.io/?source=ARS&id=5375a9be-...,https://arax.ncats.io/?source=ARS&id=24ad43ed-...,https://arax.ncats.io/?source=ARS&id=d97ca0f0-...,...,https://arax.ncats.io/?source=ARS&id=5b31d4f6-...,https://arax.ncats.io/?source=ARS&id=1a97f2b9-...,https://arax.ncats.io/?source=ARS&id=3fc14f87-...,https://arax.ncats.io/?source=ARS&id=8c2558d9-...,https://arax.ncats.io/?source=ARS&id=86623b97-...,https://arax.ncats.io/?source=ARS&id=3e00623c-...,https://arax.ncats.io/?source=ARS&id=9e9ee77c-...,https://arax.ncats.io/?source=ARS&id=62edaf1f-...,https://arax.ncats.io/?source=ARS&id=464d9ca4-...,https://arax.ncats.io/?source=ARS&id=261b17dd-...
kp-chp,No Results,No Results,No Results,Results,No Results,No Results,Error,Error,Error,Error,...,Results,No Results,Error,No Results,No Results,No Results,No Results,No Results,No Results,No Results
ara-unsecret,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,...,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error
kp-genetics,No Results,No Results,Unknown,No Results,No Results,No Results,Unknown,Unknown,Unknown,Unknown,...,No Results,No Results,No Results,Unknown,Unknown,Unknown,No Results,Unknown,Unknown,Unknown
kp-textmining,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,...,No Results,No Results,Error,Error,No Results,Error,No Results,Error,Error,Error
kp-icees,No Results,No Results,ARS Error,No Results,No Results,No Results,ARS Error,ARS Error,ARS Error,ARS Error,...,No Results,No Results,No Results,ARS Error,ARS Error,ARS Error,No Results,ARS Error,ARS Error,ARS Error
ara-explanatory,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,...,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results
ara-improving,Results,Results,Results,Results,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,...,ARS Error,No Results,ARS Error,Error,Results,Results,Results,Results,Results,Results
ara-arax,No Results,Results,Results,Results,Results,Results,Error,No Results,Error,No Results,...,Results,Results,No Results,Error,Results,Results,Results,Error,Error,Error
ara-aragorn,No Results,Results,Results,Results,Error,Error,Error,No Results,No Results,No Results,...,Results,Results,No Results,Error,Results,Results,Results,No Results,Error,No Results


<br>

## Converting the Pk's to hyperlink

<br>

In [7]:
df.loc['pk_id'] = df.loc['pk_id'].apply(lambda x: make_hyperlink(x))

In [8]:
df.rename({'pk_id': 'pk'}, inplace=True)

In [9]:
df.sort_index(inplace=True)

In [10]:
df

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1d_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,...,B.3_DILI-one-hop-from-genes,B.4_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,C.2_ChemSubstances_related_to_GeneSet,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine-with-curie,D.5_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,No Results,Results,Results,Results,Error,Error,Error,No Results,No Results,No Results,...,Results,Results,No Results,Error,Results,Results,Results,No Results,Error,No Results
ara-arax,No Results,Results,Results,Results,Results,Results,Error,No Results,Error,No Results,...,Results,Results,No Results,Error,Results,Results,Results,Error,Error,Error
ara-bte,No Results,Results,No Results,Results,No Results,No Results,No Results,No Results,No Results,No Results,...,Results,No Results,Error,Error,Error,Error,Error,Error,Error,Error
ara-explanatory,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,...,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results
ara-improving,Results,Results,Results,Results,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,...,ARS Error,No Results,ARS Error,Error,Results,Results,Results,Results,Results,Results
ara-unsecret,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,...,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error
kp-cam,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,...,No Results,Results,Error,Error,Error,Error,Error,Error,Error,Error
kp-chp,No Results,No Results,No Results,Results,No Results,No Results,Error,Error,Error,Error,...,Results,No Results,Error,No Results,No Results,No Results,No Results,No Results,No Results,No Results
kp-cohd,Error,Error,Error,ARS Error,Error,Error,Error,Error,Error,Error,...,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error
kp-genetics,No Results,No Results,Unknown,No Results,No Results,No Results,Unknown,Unknown,Unknown,Unknown,...,No Results,No Results,No Results,Unknown,Unknown,Unknown,No Results,Unknown,Unknown,Unknown


<br>

### Highlight the cells In Excel

<br>

In [11]:
def highlight(v):
    if v=='Results':
        return 'background-color: %s' % 'green'
    elif v=='Error':
        return 'background-color: %s' % 'red'
    elif v == 'No Results':
        return 'background-color: %s' % 'yellow'
    elif v == 'ARS Error':
        return 'background-color: %s' % 'blue'
    elif v == 'Unknown':
        return 'background-color: %s' % 'magneta'
    else:
        return

In [12]:
styled = df.style.applymap(highlight)

In [13]:
styled

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1d_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1e_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1f_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.2_DILI-fourth-one-hop-from-disease-or-phenotypic-feature_trapi,B.3_DILI-one-hop-from-genes,B.4_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,C.2_ChemSubstances_related_to_GeneSet,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine-with-curie,D.5_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,No Results,Results,Results,Results,Error,Error,Error,No Results,No Results,No Results,No Results,No Results,Error,Results,Results,No Results,Error,Results,Results,Results,No Results,Error,No Results
ara-arax,No Results,Results,Results,Results,Results,Results,Error,No Results,Error,No Results,Error,No Results,Error,Results,Results,No Results,Error,Results,Results,Results,Error,Error,Error
ara-bte,No Results,Results,No Results,Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,Results,No Results,Error,Error,Error,Error,Error,Error,Error,Error
ara-explanatory,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,Error,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results
ara-improving,Results,Results,Results,Results,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,Error,ARS Error,No Results,ARS Error,Error,Results,Results,Results,Results,Results,Results
ara-unsecret,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error
kp-cam,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,Results,Error,Error,Error,Error,Error,Error,Error,Error
kp-chp,No Results,No Results,No Results,Results,No Results,No Results,Error,Error,Error,Error,Error,Error,Error,Results,No Results,Error,No Results,No Results,No Results,No Results,No Results,No Results,No Results
kp-cohd,Error,Error,Error,ARS Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error
kp-genetics,No Results,No Results,Unknown,No Results,No Results,No Results,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Error,No Results,No Results,No Results,Unknown,Unknown,Unknown,No Results,Unknown,Unknown,Unknown


In [ ]:
date = datetime.now().strftime("%Y_%m_%d-%I_%M_%S_%p")
wks_name = 'Workflow Progress Tracker_' + date

In [14]:
styled.to_excel('/Users/priyash/Documents/GitHub/minihackathons/Notebooks/' + wks_name + '_' + '.xlsx')

/usr/local/lib/python3.9/site-packages/pandas/io/formats/excel.py:375: CSSWarning: Unhandled color format: 'magneta'
  warnings.warn(f"Unhandled color format: {repr(val)}", CSSWarning)


In [ ]:
### Save as excel sheet
#df.to_excel('ara_worklow_progress_tracker_Aug5.xlsx', sheet_name = 'Progress_Tracker')

In [ ]:
### save all the ids for ara submission
#with open('ara_worklow_IDs_Aug9.txt', 'w') as f:
#    print(dict_workflows, file=f)

<br>

### Pushing dataframe to excel sheet on google drive

<br>


**Here I am using the google drive API to push the daatframe into an axcel sheet 
Every individula has the unique credential file that they need to create for google drive API -- 
"araxworkflowprogresstesting-2632632db8be.json" -- is the credential used from my drive. place this json file where
the ReadAndRunAllWorkFLows.ipynb will be. NB: i have removed my credntial file for privacy reasons. Always remove
the json file before making committs to the repo. To use googe Drive API follow: https://towardsdatascience.com/how-to-manage-files-in-google-drive-with-python-d26471d91ecd**

<br>

In [15]:
styled

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1d_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1e_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1f_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.2_DILI-fourth-one-hop-from-disease-or-phenotypic-feature_trapi,B.3_DILI-one-hop-from-genes,B.4_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,C.2_ChemSubstances_related_to_GeneSet,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine-with-curie,D.5_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,No Results,Results,Results,Results,Error,Error,Error,No Results,No Results,No Results,No Results,No Results,Error,Results,Results,No Results,Error,Results,Results,Results,No Results,Error,No Results
ara-arax,No Results,Results,Results,Results,Results,Results,Error,No Results,Error,No Results,Error,No Results,Error,Results,Results,No Results,Error,Results,Results,Results,Error,Error,Error
ara-bte,No Results,Results,No Results,Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,Results,No Results,Error,Error,Error,Error,Error,Error,Error,Error
ara-explanatory,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,Error,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results
ara-improving,Results,Results,Results,Results,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,Error,ARS Error,No Results,ARS Error,Error,Results,Results,Results,Results,Results,Results
ara-unsecret,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error
kp-cam,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,Results,Error,Error,Error,Error,Error,Error,Error,Error
kp-chp,No Results,No Results,No Results,Results,No Results,No Results,Error,Error,Error,Error,Error,Error,Error,Results,No Results,Error,No Results,No Results,No Results,No Results,No Results,No Results,No Results
kp-cohd,Error,Error,Error,ARS Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error
kp-genetics,No Results,No Results,Unknown,No Results,No Results,No Results,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Error,No Results,No Results,No Results,Unknown,Unknown,Unknown,No Results,Unknown,Unknown,Unknown


In [16]:
df

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1d_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,...,B.3_DILI-one-hop-from-genes,B.4_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,C.2_ChemSubstances_related_to_GeneSet,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine-with-curie,D.5_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,No Results,Results,Results,Results,Error,Error,Error,No Results,No Results,No Results,...,Results,Results,No Results,Error,Results,Results,Results,No Results,Error,No Results
ara-arax,No Results,Results,Results,Results,Results,Results,Error,No Results,Error,No Results,...,Results,Results,No Results,Error,Results,Results,Results,Error,Error,Error
ara-bte,No Results,Results,No Results,Results,No Results,No Results,No Results,No Results,No Results,No Results,...,Results,No Results,Error,Error,Error,Error,Error,Error,Error,Error
ara-explanatory,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,...,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results
ara-improving,Results,Results,Results,Results,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,...,ARS Error,No Results,ARS Error,Error,Results,Results,Results,Results,Results,Results
ara-unsecret,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,...,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error
kp-cam,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,...,No Results,Results,Error,Error,Error,Error,Error,Error,Error,Error
kp-chp,No Results,No Results,No Results,Results,No Results,No Results,Error,Error,Error,Error,...,Results,No Results,Error,No Results,No Results,No Results,No Results,No Results,No Results,No Results
kp-cohd,Error,Error,Error,ARS Error,Error,Error,Error,Error,Error,Error,...,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error
kp-genetics,No Results,No Results,Unknown,No Results,No Results,No Results,Unknown,Unknown,Unknown,Unknown,...,No Results,No Results,No Results,Unknown,Unknown,Unknown,No Results,Unknown,Unknown,Unknown


<br>

## Google Drive API

**Push the dataframe to a google sheet via google drive API and then format the google spread sheet to add hyperlink to pk's and color the cells

<br>

In [17]:
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials

In [18]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'araxworkflowprogresstesting-2632632db8be.json', scope)
gc = gspread.authorize(credentials)

In [19]:
spreadsheet_key = '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A'
#spreadsheet_key = '1sPpBIkxrHbQNiTm5oPs9-5KrjsyXcgaVAxknJj-u8pY'
wks_name = 'Workflow Progress Tracker' + date
d2g.upload(df, spreadsheet_key, wks_name, credentials=credentials, row_names=True)

<Worksheet 'Workflow Progress Tracker August 20' id:1770348413>

In [20]:
spreadsheet_key

'1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A'

In [21]:
import gspread
import pandas as pd
from gspread_dataframe import set_with_dataframe
from gspread_formatting import *

In [ ]:
pwd

In [44]:
gc = gspread.service_account(filename='/Users/priyash/Documents/GitHub/minihackathons/Notebooks/araxworkflowprogresstesting-2632632db8be.json')
wksh = gc.open("workflow_progress_tracker")
sh = wksh.worksheet('Workflow Progress Tracker August 20')

In [23]:
sh

<Worksheet 'Workflow Progress Tracker August 20' id:1770348413>

In [24]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('B2:X16', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_EQ', ['Error']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(1,0,0))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{}]}

In [25]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('B2:X16', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_EQ', ['Results']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.0, 0.5, 0.0))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1770348413,
       'startRowIndex': 1,
       'endRowIndex': 16,
       'startColumnIndex': 1,
       'endColumnIndex': 24}],
     'booleanRule': {'condition': {'type': 'TEXT_EQ',
       'values': [{'userEnteredValue': 'Error'}]},
      'format': {'backgroundColor': {'red': 1},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'red': 1}}}}}}},
  {},
  {}]}

In [26]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('B2:X15', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_EQ', ['No Results']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.75, 0.75, 0))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1770348413,
       'startRowIndex': 1,
       'endRowIndex': 16,
       'startColumnIndex': 1,
       'endColumnIndex': 24}],
     'booleanRule': {'condition': {'type': 'TEXT_EQ',
       'values': [{'userEnteredValue': 'Results'}]},
      'format': {'backgroundColor': {'green': 0.49803922},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'green': 0.49803922}}}}}}},
  {'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1770348413,
       'startRowIndex': 1,
       'endRowIndex': 16,
       'startColumnIndex': 1,
       'endColumnIndex': 24}],
     'booleanRule': {'condition': {'type': 'TEXT_EQ',
       'values': [{'userEnteredValue': 'Error'}]},
      'format': {'backgroundColor': {'red': 1},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'red': 1}}}}}}},
  {},
  {

In [27]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('B2:X15', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_EQ', ['ARS Error']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.0, 0.75, 0.75))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1770348413,
       'startRowIndex': 1,
       'endRowIndex': 15,
       'startColumnIndex': 1,
       'endColumnIndex': 24}],
     'booleanRule': {'condition': {'type': 'TEXT_EQ',
       'values': [{'userEnteredValue': 'No Results'}]},
      'format': {'backgroundColor': {'red': 0.7490196, 'green': 0.7490196},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'red': 0.7490196,
         'green': 0.7490196}}}}}}},
  {'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1770348413,
       'startRowIndex': 1,
       'endRowIndex': 16,
       'startColumnIndex': 1,
       'endColumnIndex': 24}],
     'booleanRule': {'condition': {'type': 'TEXT_EQ',
       'values': [{'userEnteredValue': 'Results'}]},
      'format': {'backgroundColor': {'green': 0.49803922},
       'textFormat': {'bold': True},
       'back

In [28]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('B2:X15', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_EQ', ['Unknown']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.75, 0, 0.75))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1770348413,
       'startRowIndex': 1,
       'endRowIndex': 15,
       'startColumnIndex': 1,
       'endColumnIndex': 24}],
     'booleanRule': {'condition': {'type': 'TEXT_EQ',
       'values': [{'userEnteredValue': 'ARS Error'}]},
      'format': {'backgroundColor': {'green': 0.7490196, 'blue': 0.7490196},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'green': 0.7490196,
         'blue': 0.7490196}}}}}}},
  {'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1770348413,
       'startRowIndex': 1,
       'endRowIndex': 15,
       'startColumnIndex': 1,
       'endColumnIndex': 24}],
     'booleanRule': {'condition': {'type': 'TEXT_EQ',
       'values': [{'userEnteredValue': 'No Results'}]},
      'format': {'backgroundColor': {'red': 0.7490196, 'green': 0.7490196},
       'textFormat': {'bold'

In [29]:
# Select a range
cell_list = sh.range('B16:X16')

# Update in batch
sh.update_cells(cell_list,value_input_option='USER_ENTERED')

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'updatedRange': "'Workflow Progress Tracker August 20'!B16:X16",
 'updatedRows': 1,
 'updatedColumns': 23,
 'updatedCells': 23}

In [ ]:
df

In [50]:
cell_list = sh.findall('Unknown')

In [51]:
cell_list

[<Cell R11C4 'Unknown'>,
 <Cell R11C8 'Unknown'>,
 <Cell R11C9 'Unknown'>,
 <Cell R11C10 'Unknown'>,
 <Cell R11C11 'Unknown'>,
 <Cell R11C12 'Unknown'>,
 <Cell R11C13 'Unknown'>,
 <Cell R11C18 'Unknown'>,
 <Cell R11C19 'Unknown'>,
 <Cell R11C20 'Unknown'>,
 <Cell R11C22 'Unknown'>,
 <Cell R11C23 'Unknown'>,
 <Cell R11C24 'Unknown'>]

In [47]:
cell_list = sh.range('B16:X16')

In [48]:
cell_list

[<Cell R16C2 'https://arax.ncats.io/?source=ARS&id=28e96df2-53ca-491f-a468-b1a5de653473'>,
 <Cell R16C3 'https://arax.ncats.io/?source=ARS&id=f1695a43-c421-4499-a8e3-ab04627a8959'>,
 <Cell R16C4 'https://arax.ncats.io/?source=ARS&id=8c63ec23-8d39-4c0f-9b01-18495b1df83b'>,
 <Cell R16C5 'https://arax.ncats.io/?source=ARS&id=e585225c-c201-46e1-8798-f3d5da8d5bc0'>,
 <Cell R16C6 'https://arax.ncats.io/?source=ARS&id=56f06fb4-315a-4d4b-a7f7-5a18e7119b65'>,
 <Cell R16C7 'https://arax.ncats.io/?source=ARS&id=0e016263-8d0e-4375-9db7-576c3a6e02f6'>,
 <Cell R16C8 'https://arax.ncats.io/?source=ARS&id=c6856c98-b05f-4e5f-8f5a-7b6433dfd6b3'>,
 <Cell R16C9 'https://arax.ncats.io/?source=ARS&id=5375a9be-a284-407f-bbfd-eb564e8ea655'>,
 <Cell R16C10 'https://arax.ncats.io/?source=ARS&id=24ad43ed-5b5b-4b4d-bb84-d7958208af5a'>,
 <Cell R16C11 'https://arax.ncats.io/?source=ARS&id=d97ca0f0-1c11-45ba-9d0b-c4732f35df59'>,
 <Cell R16C12 'https://arax.ncats.io/?source=ARS&id=eb3304b1-f3d1-4782-b29a-f1698cb170ec

In [49]:
pwd

'/Users/priyash/Documents/GitHub/minihackathons/Notebooks'

In [63]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('B2:{}16', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_EQ', ['Error']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(1,0,0))
    )
)